**Rationale** Hypothesis testing is foundational to the entire field of statistics. Without the Central Limit Theorem and hypothesis testing, we would not have modern day (social) science. In marketing, this is generally most important when doing A/B testing of ads. In this assignment, you will practice computing basic summary statistics and conducting hypothesis tests. 

[Datasets](https://drive.google.com/drive/folders/1D6D-zRU3oiP12c0dFMQjs66JDhBZBtax?usp=sharing) required:
1. [FB ad campaign data](https://drive.google.com/file/d/1JfwvumxS2oys8ZcQtsb8GGmeRfnkzFdy/view?usp=sharing).
1. [Starbucks Promos](https://drive.google.com/file/d/1W7x5_PU5KszT8mqXqqEmRfXuR8CRr1DX/view?usp=sharing)

# 1. (2 points) FB Ad campaigns

1. Use a groupby operation create a dataframe called `sumstats` consisting of the total impressions and clicks for each `xyz_campaign_id`.
1. Using `sumstats`, create a new column called `ctr` that represents the [click through rate](https://support.google.com/google-ads/answer/2615875?hl=en) for each campaign. Which campaign had the highest CTR?
1. Create a column for `sd` for standard deviation of the click through rate. 

Hint: Recall from the notes that the standard deviation of a binomial distribution with $Pr(Success) = p$ is $SD(p) = \sqrt{p(1-p)}$. So if the probability of a binary outcome (such as clicking on an ad) is observed to be .3, the standard deviation is $\sqrt{.3(.7)}=\sqrt{.21}$.

In [2]:
# imports and mount google drive
import os, pandas as pd, numpy as np
from scipy import stats



from google.colab import drive
drive.mount('drive')

Mounted at drive


In [3]:
# set the path to the datasets for A5
fpath = 'drive/MyDrive/Module5/'
os.listdir(fpath)

['Copy of facebook_ads.gsheet',
 'Copy of starbucks_promos.gsheet',
 'facebook_ads.csv',
 'starbucks_promos.csv']

In [4]:
# read in the facebook_ads.csv file as the dataframe variable named fb
fb = pd.read_csv(fpath + 'facebook_ads.csv')

In [5]:
# Take a look at the first 5 rows
# Observe that each row represents 1 ad from 1 campaign and a target demographic group (age, gender, interest)
fb.head()

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
0,708746,916,103916,30-34,M,15,7350,1,1.43,2,1
1,708749,916,103917,30-34,M,16,17861,2,1.82,2,0
2,708771,916,103920,30-34,M,20,693,0,0.00,1,0
3,708815,916,103928,30-34,M,28,4259,1,1.25,1,0
4,708818,916,103928,30-34,M,28,4133,1,1.29,1,1


In [6]:
# use a groupby to create new dataframe, sumstats that tabulates the total clicks and impressions for each campaign.
# Generic groupby syntax: sumstats = df.groupby('groupbyvariable')[['summary_variable1', 'summary_variable2']].summaryfunction().reset_index()
# use the correct dataframe name, variables (i.e. column names) and summaryfunction

sumstats= fb.groupby('xyz_campaign_id')[['Clicks','Impressions']].sum().reset_index()

In [7]:
sumstats

,xyz_campaign_id,Clicks,Impressions
0,916,113,482925
1,936,1984,8128187
2,1178,36068,204823716


In [8]:
# create a new column in sumstats called 'ctr' (short for click through rate)
# recall that ctr = clicks / impressions

sumstats['ctr']=sumstats['Clicks']/sumstats['Impressions']


In [9]:
# take a look at the whole sumstats dataframe

sumstats

,xyz_campaign_id,Clicks,Impressions,ctr
0,916,113,482925,0.000234
1,936,1984,8128187,0.000244
2,1178,36068,204823716,0.000176


In [10]:
# create a new column ctr_sd for the std. dev of the ctr (See hint above)
# pay attention to PEMDAS, esp. that you want to take the sqrt after computing p*(1-p)
# np.sqrt(x) returns the sqrt of some number x
# alternatively, x**.5 also returns the sqrt of x
sumstats['ctr_sd']=np.sqrt(sumstats.ctr * (1-sumstats.ctr))


In [11]:
# take another look at the whole sumstats dataframe
# compare the ctr_std to calculating it by hand, make sure you did this correctly.

sumstats

,xyz_campaign_id,Clicks,Impressions,ctr,ctr_sd
0,916,113,482925,0.000234,0.015295
1,936,1984,8128187,0.000244,0.015621
2,1178,36068,204823716,0.000176,0.013269


## Answer (edit this cell)
The campaign with the highest CTR is: **__campaign ID 916__**


# 2. (4 points) Hypothesis testing and confidence intervals

1. Compute the 95% confidence interval for the CTR for each campaign. Compare the confidence intervals of campaign 916 and 936. What can you conclude about the relative perfrmance of the 2 ads in the population (e.g. are they very different, similar, etc.)? How about 916 vs. 1178?
1. Was campaign 936 statistically different compared to campaign 916? How about 936 vs. 1178? Use the `ttest_2sample` function from the notes. Remember, you must define it in your Colab session in order to use it (execute the cell w/ the function).

Given these statistical tests what would you recommend in terms of allocation of the ad budget? 

In [12]:
Mean916 = sumstats.loc[sumstats.xyz_campaign_id==916, 'ctr']

In [13]:
Mean936 = sumstats.loc[sumstats.xyz_campaign_id==936, 'ctr']

In [14]:
Mean1178 = sumstats.loc[sumstats.xyz_campaign_id==1178, 'ctr']

In [15]:
Std916 = sumstats.loc[sumstats.xyz_campaign_id==916, 'ctr_sd']

In [16]:
Std936 = sumstats.loc[sumstats.xyz_campaign_id==936, 'ctr_sd']

In [17]:
Std1178 = sumstats.loc[sumstats.xyz_campaign_id==1178, 'ctr_sd']

In [18]:
obs916 = sumstats.loc[sumstats.xyz_campaign_id==916, 'Impressions']

In [19]:
obs936 = sumstats.loc[sumstats.xyz_campaign_id==936, 'Impressions']

In [20]:
obs1178 = sumstats.loc[sumstats.xyz_campaign_id==1178, 'Impressions']

In [21]:
# use stats.norm.ppf([.025, .975], mean, std error)
# where mean is sample mean and std error is std dev / sqrt(obs) (from Central Limit Theorem)

# confidence interval for ad 916
stats.norm.ppf([.025, .975], Mean916, Std916/obs916**0.5)


array([0.00019085, 0.00027713])

In [22]:
# confidence interval for ad 936

stats.norm.ppf([.025, .975], Mean936, Std936/obs936**0.5)

array([0.00023335, 0.00025483])

In [23]:
# confidence interval for ad 1178
stats.norm.ppf([.025, .975], Mean1178, Std1178/obs1178**0.5)


array([0.00017428, 0.00017791])

**Edit this cell**

1. (write 1-2 sentences to compare the confidence interval of 916 vs 936's CTR)
Campaign 936 has a higher mean than 916, meaning the data is more spread out in 936 than 916. 
1. (write 1-2 sentences to compare the confidence interval of 916 vs 1178's CTR)
Campaign 916 has a large spread of data compared to 1178.

In [24]:
# Use this custom funciton

def ttest_2sample(m1,sd1,N1,m2,sd2,N2, twotail = True, equalvar = False):
    """
    This function requires you tu supply:
    m1: mean of sample 1
    sd1: std. dev of sample 1
    N1: number of obs of sample 1
    m2: mean of sample 2
    sd2: std dev of sample 2
    N2: number of obs of sample 2

    Optional inputs:
    twotail = True (default) / False. If False, then 1 tail
    equalvar = True / False (default). If True, assumes equal population variance.
    """

    # The difference between equal and unequal variance is only in how to compute
    # the test statistic and degree of freedom.
    if equalvar:
        spsquare = ((N1-1)*sd1**2+(N2-1)*sd2**2)/(N1+N2-2) 
        T = (m1-m2)/np.sqrt(spsquare*(1/N1+1/N2))
        nu = N1+N2-2
    else:
        nu = (sd1**2/N1+sd2**2/N2)**2/((sd1**2/N1)**2/(N1-1)+(sd2**2/N2)**2/(N2-1)) # new degree of freedom
        T = (m1-m2)/(np.sqrt(sd1**2/N1+sd2**2/N2))
    
    # If the first mean is bigger, we need to do 1- cdf
    # Otherwise just compute cdf
    if m1>m2:
        pval = 1-stats.t.cdf(T, df = nu)
    else:
        pval = stats.t.cdf(T, df = nu)

    # return p values
    # If 2 tail, we must multiply by 2
    # otherwise we just return the computed pval
    if twotail == True:
        return pval*2
    else:
        return pval

In [25]:
Mean916

0    0.000234
Name: ctr, dtype: float64

In [26]:
Std916

0    0.015295
Name: ctr_sd, dtype: float64

In [27]:
obs916

0    482925
Name: Impressions, dtype: int64

In [28]:
Mean936

1    0.000244
Name: ctr, dtype: float64

In [29]:
Std936

1    0.015621
Name: ctr_sd, dtype: float64

In [30]:
obs936

1    8128187
Name: Impressions, dtype: int64

In [31]:
Mean1178

2    0.000176
Name: ctr, dtype: float64

In [32]:
Std1178

2    0.013269
Name: ctr_sd, dtype: float64

In [33]:
obs1178

2    204823716
Name: Impressions, dtype: int64

In [34]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 916
m1,sd1,obs1 = .000234, .015295, 482925
m2,sd2,obs2 = .000244, .015621, 8128187

ttest_2sample(m1,sd1,obs1,m2,sd2,obs2, twotail = True, equalvar = False)

0.659290373515292

In [35]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 1178
m2,sd2,obs2 = .000244, .015621, 8128187
m3,sd3,obs3 = .000176, .013269, 204823716

ttest_2sample(m2,sd2,obs2,m3,sd3,obs3, twotail = True, equalvar = False)

0.0

**Edit this cell to answer**

(Write a few sentences to describe how to proceed with the allocation of the ad budget going forward. Be sure to use the 2 tail t-tests above to support your strategy)

There is 66% difference between 936 and 916. That uncertainty means there needs to less of a scattered population and rejecting thr null hypothesis
In the second test, the null hypothesis is accepted at 0 as there is no uncertainty.

# 3. (4 points)

Using the Starbucks dataset, conduct an appropriate statistical test to determine:
1. Is there any statistically appreciable difference in the redemption rates of BOGO vs. Discount promotions?
1. Among those who redeemed an offer and reported their income, is there any statistical difference in the average income associated with BOGO vs. discount redemptions? Note that not every observation reports income (select only the observations that do).

In [36]:
# set the path to the datasets for A5
fpath = 'drive/MyDrive/Module5/'
os.listdir(fpath)

['Copy of facebook_ads.gsheet',
 'Copy of starbucks_promos.gsheet',
 'facebook_ads.csv',
 'starbucks_promos.csv']

In [37]:
# read in the starbucks_promos.csv file as the dataframe variable named sb, use index_col = 0 option
sb = pd.read_csv(fpath + 'starbucks_promos.csv', index_col= 0)

In [38]:
# preview first 5 rows to get a sense of the contents of the dataframe
# note that each observation represents an offer received by a customer
sb.head()

,uid,event,time,gender,age,register_date,income,offer_id,offer_reward,channels,difficulty,duration,offer_type,offer_time,transaction_amount,redeem_time,redeemed
1,0020c2b971eb4e9188eac86d93036a77,offer received,0,F,59,20160304,90000.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,"['web', 'email', 'mobile', 'social']",10.0,240.0,discount,0.0,17.63,54.0,1
4,005500a7188546ff8a767329a2f7c76a,offer received,0,M,56,20171209,47000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,NaN,NaN,0
5,0056df74b63b4298809f0b375a304cf4,offer received,0,M,54,20160821,91000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"['web', 'email', 'mobile']",5.0,168.0,bogo,0.0,27.86,132.0,1
6,00715b6e55c3431cb56ff7307eb19675,offer received,0,F,58,20171207,119000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,27.26,12.0,1
8,00840a2ca5d2408e982d56544dc14ffd,offer received,0,M,26,20141221,61000.0,2906b810c7d4411798c6938adc9daaa5,2.0,"['web', 'email', 'mobile']",10.0,168.0,discount,0.0,6.05,540.0,1


In [39]:
# select the reemed column for discount offers, name this data the variable disc

disc = sb.loc[sb.offer_type=='discount', ['redeemed']]

disc

,redeemed
1,1
8,1
13,1
17,1
19,0
...,...
76259,1
76264,1
76266,1
76272,0


In [40]:
# select the reemed column for buy one get one (bogo) offers, name this data the variable bogo
bogo = sb.loc[sb.offer_type=='bogo', ['redeemed']]

bogo

,redeemed
4,0
5,1
6,1
9,0
10,0
...,...
76269,0
76271,1
76273,1
76274,0


In [41]:
# use stats.ttest_ind to test whether the 2 types of offers yield different redemption rates

stats.ttest_ind(disc, bogo,equal_var=False)

Ttest_indResult(statistic=array([17.30134897]), pvalue=array([6.64177493e-67]))

In [43]:
# Select the income column for those who redeemed the BOGO offer, call this the variable bogo_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for bogo offer, and one for income is not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data

bogo_income = sb.loc[(sb.redeemed==1)&(sb.offer_type == 'bogo')&(sb.income.notnull()), 'income']

bogo_income


5         91000.0
6        119000.0
11        81000.0
23        80000.0
24       102000.0
           ...   
76263     92000.0
76268     77000.0
76271     93000.0
76273     73000.0
76275     34000.0
Name: income, Length: 15933, dtype: float64

In [47]:
# Select the income column for those who redeemed the discount offer, call this the variable disc_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for discount offer,  and one for income is not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data

disc_income = sb.loc[(sb.redeemed==1)&(sb.offer_type == 'discount')&(sb.income.notnull()), 'income']

disc_income

1         90000.0
8         61000.0
13        66000.0
17       102000.0
21        72000.0
           ...   
76256     53000.0
76259     86000.0
76264     32000.0
76266     47000.0
76276     62000.0
Name: income, Length: 17710, dtype: float64

In [48]:
# What is the average income for bogo redeemers (bogo_income.mean())?

bogo_income.mean()

70114.8559593297

In [49]:
# What is the average income for discount redeemers?

disc_income.mean()

68617.39130434782

In [50]:
# use stats.ttest_ind to test whether the average income of the redeemers of the 2 offer types are different

stats.ttest_ind(bogo_income, disc_income, equal_var=False)

Ttest_indResult(statistic=6.342911358743618, pvalue=2.283574816464658e-10)

**Edit this cell**
1. (Write a sentence to describe the relative redemption rates of the 2 offer types based on the result of the t-test).
Individuals who receive redemption rates usually have an everage income between $68-69,000
1. (Write a sentence to describe the relative incomes of the redeemers of the 2 offer types based on the result of the t-test).
The probablity of whether the incomes are different is indicated by the P-value of 2.28. 